In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import torch

from transformers import PretrainedConfig
from model.visual_token_embedding import VisualTokenEmbedding


In [ ]:
visual_token_embedding_config = PretrainedConfig.from_dict({
    "image_resolution": 768,
    "token_resolution": 8,

    # "vision_encoder_type": "rgb_pixel",
    # "vision_encoder_name": "rgb_pixel",
    
    "vision_encoder_type": "diffusers_vae",
    "vision_encoder_name": "chendelong/stable-diffusion-3-medium-vae",

    # https://huggingface.co/models?sort=trending&search=facebook%2Fdinov2
    # "vision_encoder_type": "hf_autobacbone",
    # "vision_encoder_name": "facebook/dinov2-giant", # small, base, large, giant
    
    # https://huggingface.co/models?search=facebook/convnextv2
    # "vision_encoder_type": "hf_autobacbone",
    # "vision_encoder_name": "facebook/convnextv2-tiny-22k-384/stage2", 

    # # https://huggingface.co/models?search=microsoft/resnet
    # "vision_encoder_type": "hf_autobacbone",
    # "vision_encoder_name": "microsoft/resnet-50", # 18, 34, 50, 101

    # https://huggingface.co/timm
    # "vision_encoder_type": "timm_backbone",
    # "vision_encoder_name": "convnext_large_mlp.clip_laion2b_soup_ft_in12k_in1k_320/-2", 
})


visual_token_embedding = VisualTokenEmbedding(visual_token_embedding_config).cuda()
print(visual_token_embedding.device, visual_token_embedding.dtype)

print(visual_token_embedding.vision_encoder.feature_channels, 'channels')

In [ ]:
# get an image
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")

feature_maps = visual_token_embedding.vision_encoder([image])
print(feature_maps.shape, feature_maps.dtype, feature_maps.device)
feature_maps = feature_maps.cpu().numpy()

In [ ]:
from sklearn.decomposition import PCA
import numpy as np

if feature_maps.shape[1] > 3:
    print(feature_maps.shape)
    n_components = 3

    X = np.swapaxes(feature_maps, -1, 1)
    X = np.reshape(X, (-1, X.shape[-1]))

    pca = PCA(n_components=n_components)
    pca.fit(X)
    projection = pca.components_ # n_components, feature_dim

    # project the features to the PCA space -> n_images, n_components, h, w
    feature_maps = np.matmul(np.swapaxes(feature_maps, -1, 1), projection.T)
    feature_maps = np.swapaxes(feature_maps, -1, 1)

    # normalize the features to [0, 1]
    feature_maps = (feature_maps - np.min(feature_maps)) / (np.max(feature_maps) - np.min(feature_maps))

    print(feature_maps.shape)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 10))

plt.subplot(1, 2, 1)
plt.imshow(image.resize((512, 512)))
plt.axis("off")

plt.subplot(1, 2, 2)
feature_maps_rgb = np.transpose(feature_maps[0], (1, 2, 0))
plt.imshow(feature_maps_rgb)
plt.axis("off")

plt.show()
